# Network Intrusion Detection with Deep Learning

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


## Open Data

In [0]:
import numpy as np
def load_matrices(file_name):
    with open(file_name, 'rb') as f:
        A = np.load(f)
        B = np.load(f)
        C = np.load(f)
        D = np.load(f)
        E = np.load(f)
        F = np.load(f)
    return (A,B,C,D,E,F)

In [0]:
my_file = '../data_NSL/data.npy'
train, test, test2, y_train, y_test, y_test2 = load_matrices(my_file)

## The Model

In [8]:
# talos item
import talos as ta
import wrangle as wr
from talos.metrics.keras_metrics import fmeasure_acc
from talos import live

# Keras items
from keras import backend as K
from keras.layers import Input, Dense 
from keras.layers import InputLayer
from keras.models import Model, Sequential
from keras.layers import Activation, Reshape, Dropout
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping
from keras.activations import relu, elu
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import PReLU
from keras.callbacks import TensorBoard
from keras.models import load_model

%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [0]:
# We use early stopping on a holdout validation set

In [0]:
def selu(x):
    """Scaled Exponential Linear Unit. (Klambauer et al., 2017)
    # Arguments
        x: A tensor or variable to compute the activation function for.
    # References
        - [Self-Normalizing Neural Networks](https://arxiv.org/abs/1706.02515)
    """
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * elu(x, alpha)

#model.add(Dense(32, input_shape=(input_length - 1,)), activation=selu)

In [11]:
# then we load the dataset
x=train
y=y_train
# and normalize every feature to mean 0, std 1
#x = wr.mean_zero(pd.DataFrame(x)).values
input_dim = train.shape[1]
print(input_dim)

122


In [0]:
from keras.callbacks import ModelCheckpoint
filepath="./log1lr/taloslr_ae_weights_lr_pr.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:

filepath2="./log1lr/taloslr_dnn_weights_lr_pr.best.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_acc', save_best_only=True, mode='max')
callbacks_list2 = [checkpoint2]

In [0]:
#actvation leakyrelu
LR= LeakyReLU(0.01)
LR.__name__ = 'lrelu'
#actvation prelu
#PR= PReLU()
#PR.__name__ = 'prelu'

In [0]:
# first we have to make sure to input data and params into the function
# add input parameters to the function
def dnn(x_train, y_train, x_val, y_val, params):

    input_dim = x_train.shape[1]

    input_img = Input(shape=(input_dim,))

    first_neuron=params['first_neuron']
    batch_size=params['batch_size']
    kernel_initial=params['kernel_initializer']
    lr=params['lr']
    activation=params['activation']
    
    print('activation=',activation)

    ###AE
    
    # replace the hyperparameter inputs with references to params dictionary 
    autoencoder = Sequential()
    #encoder
    
    if activation==PReLU: 
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        
                   
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
        
    else:
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), activation=activation, kernel_initializer=kernel_initial))
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
    
    #model = multi_gpu_model(model, gpus=1)
    autoencoder.summary()
    print(input_dim)

    input_img = Input(shape=(input_dim,))
    encoder_layer1 = autoencoder.layers[0]
   
    encoder = Model(input_img, encoder_layer1(input_img))
    
    #encoder_layer1 = autoencoder.layers[0]
    #encoder_layer2 = autoencoder.layers[1]
    #encoder_layer3 = autoencoder.layers[2]
    #encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))
    encoder.summary()

    ## using binary cross entropy because from 122 features only 32 features (sudah dinormalisasi dalam rentang 0 dan 1) float and 90 features binary
    autoencoder.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    #model.compile(loss=params['losses'],
    #              optimizer=params['optimizer'](),
    #              metrics=['acc', fmeasure_acc])
    
   

    
    history=autoencoder.fit(x,x,
                             epochs=params['epochs'],
                             batch_size=batch_size,
                              #shuffle=True,
                              #validation_split=0.2,
                              callbacks=callbacks_list,                    
                             #callbacks=[early_stopping])
                              validation_data=(test,test),
                              verbose=0)

    num_classes=5
    out2 = Dense(num_classes, activation='softmax',kernel_initializer=kernel_initial)(encoder.output)
    newmodel = Model(encoder.input,out2)
    
   
    newmodel.compile(loss='categorical_crossentropy', 
                     #optimizer='adam', 
                     optimizer= Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=0.0, amsgrad=False),
                     metrics=['acc']) 
    newmodel.summary()


    history2a=newmodel.fit(x, y,
                           epochs=params['epochs2'],
                           batch_size=batch_size,
                           #validation_split=0.2,
                           callbacks=callbacks_list2,
                           #callbacks=[early_stopping],
                           validation_data=(test,y_test),
                           verbose=0)

    
    del encoder
    del autoencoder
    #del dnn_model
    
    tf.reset_default_graph()
    
    return history2a, newmodel #, encoder
    


In [0]:
# then we can go ahead and set the parameter space
p = {'first_neuron':[100, 80, 60,40,20],
     'batch_size': [32,64,256],
     'epochs': [30],
     'epochs2': [50],
     'dropout': [0],
     'kernel_initializer': ['lecun_uniform','lecun_normal','he_normal','he_uniform','normal','glorot_uniform','glorot_normal'],
     'optimizer': [Adam],
     #'activation':[LR]}
     'activation':[relu, elu,selu, LR,PReLU]}

In [0]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

    

In [18]:

t = ta.Scan(x, y, p, dnn, experiment_no='11',grid_downsample=0.1,)

#experiment no 6 3780 kombinasi

  0%|          | 0/52 [00:00<?, ?it/s]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x7f8572c2c3c8>
Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                2460      
_________________________________________________________________
dense_2 (Dense)              (None, 122)               2562      
Total params: 5,022
Trainable params: 5,022
Non-trainable params: 0
_________________________________________________________________
122
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 122)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2460      
Total pa

  2%|▏         | 1/52 [02:15<1:55:35, 135.99s/it]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x7f8572c2c3c8>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                2460      
_________________________________________________________________
dense_2 (Dense)              (None, 122)               2562      
Total params: 5,022
Trainable params: 5,022
Non-trainable params: 0
_________________________________________________________________
122
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 122)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2460      
Total params: 2,460
Trainable params: 2,460
Non-trainable params: 0
____________

  4%|▍         | 2/52 [06:09<2:17:36, 165.13s/it]

...

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x7f8572c2c3c8>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 40)                4920      
_________________________________________________________________
dense_2 (Dense)              (None, 122)               5002      
Total params: 9,922
Trainable params: 9,922
Non-trainable params: 0
_________________________________________________________________
122
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 122)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                4920      
Total params: 4,920
Trainable params: 4,920
Non-trainable params: 0
____________

 98%|█████████▊| 51/52 [4:28:19<04:17, 257.61s/it]

activation= <keras.layers.advanced_activations.LeakyReLU object at 0x7f8572c2c3c8>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               12300     
_________________________________________________________________
dense_2 (Dense)              (None, 122)               12322     
Total params: 24,622
Trainable params: 24,622
Non-trainable params: 0
_________________________________________________________________
122
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 122)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               12300     
Total params: 12,300
Trainable params: 12,300
Non-trainable params: 0
________

100%|██████████| 52/52 [4:33:57<00:00, 281.81s/it]

Scan Finished!


In [19]:
x.shape



(125973, 122)

In [20]:
t.data

,round_epochs,val_loss,val_acc,loss,acc,first_neuron,batch_size,epochs,epochs2,dropout,kernel_initializer,optimizer,lr,activation
1,50,1.3016710099470032,0.7114975159687722,0.22449213441013868,0.9376215538254992,20,256,30,50,0,glorot_normal,<class 'keras.optimizers.Adam'>,1e-05,<keras.layers.advanced_activations.LeakyReLU o...
2,50,2.1053323894633396,0.7957327892122072,0.02941429431493385,0.994300365951434,20,64,30,50,0,lecun_uniform,<class 'keras.optimizers.Adam'>,0.1,<keras.layers.advanced_activations.LeakyReLU o...
3,50,1.8838767722788392,0.8084190915542938,0.013153042422018493,0.9969199749152596,20,256,30,50,0,he_normal,<class 'keras.optimizers.Adam'>,0.1,<keras.layers.advanced_activations.LeakyReLU o...
4,50,1.9616250122075727,0.8146291696238467,0.004419603943548456,0.9986981337270685,100,64,30,50,0,glorot_normal,<class 'keras.optimizers.Adam'>,0.01,<keras.layers.advanced_activations.LeakyReLU o...
5,50,1.0526843610344583,0.7479151880766501,0.020197572365604148,0.9948798552070682,100,256,30,50,0,glorot_normal,<class 'keras.optimizers.Adam'>,0.0001,<keras.layers.advanced_activations.LeakyReLU o...
6,50,1.3009737995687827,0.7202803406671399,0.09513396685939829,0.968397990045486,20,64,30,50,0,lecun_normal,<class 'keras.optimizers.Adam'>,1e-05,<keras.layers.advanced_activations.LeakyReLU o...
7,50,1.2999663735383618,0.8104151880766501,0.0037699149354768844,0.9988410214887317,80,64,30,50,0,glorot_uniform,<class 'keras.optimizers.Adam'>,0.001,<keras.layers.advanced_activations.LeakyReLU o...
8,50,1.5052513296592311,0.7957327892122072,0.004642905011290637,0.9985869988013304,40,32,30,50,0,he_normal,<class 'keras.optimizers.Adam'>,0.001,<keras.layers.advanced_activations.LeakyReLU o...
9,50,2.1783524269940076,0.7848207948899929,0.07848305257216713,0.9896168226524732,40,32,30,50,0,he_normal,<class 'keras.optimizers.Adam'>,0.1,<keras.layers.advanced_activations.LeakyReLU o...
10,50,1.1990065502731238,0.7606458481192335,0.03453359408301582,0.9913949814642821,20,64,30,50,0,lecun_uniform,<class 'keras.optimizers.Adam'>,0.0001,<keras.layers.advanced_activations.LeakyReLU o...


In [0]:
r = ta.Reporting(t)

In [22]:
r.high('val_acc')

'0.823500709723208'

In [23]:
r.best_params()

array([['<keras.layers.advanced_activations.LeakyReLU object at 0x7f8572c2c3c8>',
        'normal', "<class 'keras.optimizers.Adam'>", '256', '50', '80',
        '0', '30', '0.1', 0],
       ['<keras.layers.advanced_activations.LeakyReLU object at 0x7f8572c2c3c8>',
        'he_uniform', "<class 'keras.optimizers.Adam'>", '64', '50',
        '100', '0', '30', '0.01', 1],
       ['<keras.layers.advanced_activations.LeakyReLU object at 0x7f8572c2c3c8>',
        'glorot_normal', "<class 'keras.optimizers.Adam'>", '64', '50',
        '100', '0', '30', '0.01', 2],
       ['<keras.layers.advanced_activations.LeakyReLU object at 0x7f8572c2c3c8>',
        'normal', "<class 'keras.optimizers.Adam'>", '32', '50', '60',
        '0', '30', '0.01', 3],
       ['<keras.layers.advanced_activations.LeakyReLU object at 0x7f8572c2c3c8>',
        'lecun_normal', "<class 'keras.optimizers.Adam'>", '64', '50',
        '80', '0', '30', '0.01', 4],
       ['<keras.layers.advanced_activations.LeakyReLU object 